In [1]:
import duckdb

#### Kentucky Extenxt

clipped from qgis lat/lon tool.
`-89.70443196,36.45697565,-81.88423299,39.15195113`

bbox sql:

```sql
    bbox.xmin BETWEEN -89.7 AND -81.9
    AND bbox.ymin BETWEEN 36.4 AND 39.2
```


In [2]:
# Tables where Kentucky can be queried from the table
# eliminating the need to run a spatial query on Ky's 
# border geometry.
# bbox will be used to narrow down the field of view.

# --      WHERE bbox.xmin BETWEEN -89.7 AND -81.9
# --        AND bbox.ymin BETWEEN 36.4 AND 39.2
with duckdb.connect('latest.ddb') as conn:
  conn.sql("""

    LOAD spatial;
    
    CREATE OR REPLACE TABLE place as
    WITH kybbox as (
          SELECT * FROM place
          WHERE bbox.xmin BETWEEN -89.7 AND -81.9
            AND bbox.ymin BETWEEN 36.4 AND 39.2
          )
          SELECT * from kybbox,  
            UNNEST(kybbox.addresses) as a                    
          WHERE a.unnest.region = 'KY'
    """)


  conn.sql("""

    LOAD spatial;
        
    CREATE OR REPLACE TABLE division AS
    WITH kybbox as (
          SELECT * 
          FROM division_vw
          WHERE bbox.xmin BETWEEN -89.7 AND -81.9
            AND bbox.ymin BETWEEN 36.4 AND 39.2)
          SELECT * from kybbox 
          WHERE region = 'US-KY'
      
    """)

### Clip to Kentucky Border

These tables will need further processsing.  

 - Step 1: query to bounding box
 - Step2:  clip to Kentucky boundary

In [3]:
with duckdb.connect('latest.ddb') as conn:
    conn.sql("""
        LOAD spatial;
        CREATE OR REPLACE TABLE infrastructure AS
        SELECT * 
        FROM infrastructure_vw
        WHERE bbox.xmin BETWEEN -89.7 AND -81.9
            AND bbox.ymin BETWEEN 36.4 AND 39.2
""")